In [ ]:
import numpy as np
import cv2
import os

오류는 종료 기능 없이 실행되는 코드에서 강제 종료해서 발생한 에러입니다.

## 1. 이미지 읽어오기
**alpha channel**!?  
각 화소에 대해 색상 표현의 데이터로부터 분리한 보조 데이터를 일컫는다.  
단색(RGB) 말고도 직접 표시되지 않는 색의 데이터를 넣을 수 있다.  
이것을 알파 채널이라고 부르며, 화상 마스크, 복구 화상 합성 등에 쓰인다.  

cv2.imread(img_file, cv2.IMREAD_UNCHANGED)  
- 이미지파일을 alpha channel까지 포함하여 읽어 들입니다.  

cv2.imread(img_file, cv2.IMREAD_COLOR)      
- 이미지 파일을 Color로 읽어들입니다. 투명한 부분은 무시되며, Default값입니다.

cv2.imread(img_file, cv2.IMREAD_GRAYSCALE)  
- 이미지를 Grayscale로 읽어 들입니다. 실제 이미지 처리시 중간단계로 많이 사용합니다.


In [15]:
img_file = './images/000000000731.jpg'
img_file = cv2.imread(img_file)               # opencv는 RGB가 아닌 BGR로 읽어온다.

if img_file is not None:
    print(img_file.shape)                     # height, width, channel
    cv2.imshow('0', img_file)                 # 띄우는 창 이름, 이미지 파일
    # cv2.imwrite('./images/000000000731_copy.jpg')    # 저장할 파일 경로 및 이름 지정, 파일이름
    cv2.waitKey()                             # 아무 키가 입력될 때까지 다음 이미지로 안 넘어간다.
    cv2.destroyAllWindows()                   # 이미지 다 확인 후, 창 모두 닫기
else:
    print('No image file.')

(480, 640, 3)


___________________________________________________

## 2. 동영상 및 카메라 프레임 읽기
동영상 파일이나 컴퓨터에 연결한 카메라 장치로부터 연속된 이미지 프레임을 읽을 수 있는 API를 제공한다.  

cap = cv2.VideoCapture(file_path or index): 비디오 캡처 객체 생성자
- file_path: 동영상 파일 경로
- index: 카메라 장치 번호, 0부터 순차적으로 증가
- cap: 객체

ret = cap.isOpened(): 객체 초기화 확인
- ret: 프레임 읽기 성공 또는 실패여부, True/False

ret, img = cap.read(): 영상 프레임 읽기, 튜플로 제공
- ret: 프레임 읽기 성공 또는 실패 여부, True/False
- img: 프레임 이미지, Numpy 배열 또는 None

cap.set(id, value): 속성 변경  
cap.get(id): 속성 확인  
cap.release(): 갭쳐 자원 반납(프로그램 종료하기 전에 자원을 반납해야한다)  


cv2.waitKey(25)가 필요한 이유는 각 프레임을 화면에 표시하는 시간이 너무 빠르면 우리 눈으로 볼 수 없기 때문이다.  
지연 시간은 실습에 사용할 동영상의 FPS(Frames Per Second, 초당 프레임 수)에 맞게 조정해서 적절한 속도로 영상을 재생하게 해야한다.

cv2.waitKey(): 함수의 매개변수인 숫자에서 0 은 무한대기를 가리킵니다. 만약 0 대신 아래와 같이 1000 을 입력하면 ms 단위이기 때문에 1초가 지나면 다음 라인의 소스인 cv2.destoryAllWindows() 가 실행되기 때문에 프로그램을 종료하게 됩니다.

1초를 밀리초(ms) 단위로 환산  
지연시간 = 1000 % fps  
25 = 1000 % 40

In [20]:
# 저장된 영상 불러오기
video_file = './videos/youtube.mp4'

cap = cv2.VideoCapture(video_file)
if cap.isOpened():                       # 캡쳐 객체 초기화 확인
    while True:
        ret, img = cap.read()            # 다음 프레임 읽기
        if ret:
            cv2.imshow(video_file, img)
            cv2.waitKey(25)              # 25ms 지연(40fps로 가정)
        else:
            break
else:
    print("can't open video")            # 캡쳐 객체 초기화 실패
cap.release()                            # 캡쳐 자원 반납
cv2.destroyAllWindows()

KeyboardInterrupt: 

동영상 파일과는 다르게 카메라로부터 프레임을 읽는 경우 파일의 끝이 정해져 있지 않으므로 무한루프를 빠져 나올 조건이 없습니다. 그래서 사용자가 아무 키나 누르면 빠져 나오게 했습니다.  

cv2.waitKey(): 함수는 지정한 대기 시간(1ms) 동안 키 입력이 없으면 -1을 반환한다.  
반환된 값이 -1이 아니면 아무 키나 입력되었다는 뜻.

In [19]:
# 카메라 프레임 읽기

cap = cv2.VideoCapture(0)                # 0번 카메라 장치 연결
if cap.isOpened():
    while True:
        ret, img = cap.read()            # 카메라 프레임 읽기
        if ret:
            cv2.imshow('camera', img)    # 프레임 이미지 표시
            if cv2.waitKey(1) != -1:    # 1ms 동안 키 입력대기
                break
        else:
            print('no frame')
            break
else:
    print("can't open camera.")
    
cap.release()
cv2.destroyAllWindows()

_________________________________________

### 카메라 비디오 속성 제어

get(id): 해당 속성의 값을 구할 수 있다.  
set(id, value): 아이디와 함께 값을 전달하면 값을 지정할 수 있습니다.  

속성 ID: 'cv2.CAP_PROP_'로 시작 하는 상수

cv2.CAP_PROP_FRAME_WIDTH: 프레임 폭
cv2.CAP_PROP_FRAME_HEIGHT: 프레임 높이
cv2.CAP_PROP_FPS: 초당 프레임 수
cv2.CAP_PROP_POS_MSEC: 동영상 파일의 프레임 위치(ms)
cv2.CAP_PROP_POS_AVI_RATIO: 동영상 파일의 상대 위치(0: 시작, 1:끝)
cv2.CAP_PROP_FOURCC: 동영상 파일 코덱 문자
cv2.CAP_PROP_AUTOFOCUS: 카메라 자동 초점 조절
cv2.CAP_PROP_ZOOM: 카메라 줌

**FPS 속성을 카메라 장치로부터 읽을 때는 대부분 정상적인 값을 가져오지 못한다**

In [23]:
fps = cap.get(cv2.CAP_PROP_FPS)   # 초당 프레임 수 구하기
delay = int(1000/fps)                    # 지연 시간 구하기

In [31]:
# 저장된 영상 불러오기
video_file = './videos/youtube.mp4'

cap = cv2.VideoCapture(video_file)
if cap.isOpened():                       # 캡쳐 객체 초기화 확인
    fps = cap.get(cv2.CAP_PROP_FPS)
    delay = int(1000/fps)
    print('FPS: %F, Delay: %dms'%(fps, delay))

    while True:
        ret, img = cap.read()            # 다음 프레임 읽기
        if ret:
            cv2.imshow(video_file, img)
            cv2.waitKey(delay)          # fps에 맞게 시간 지연 
        else:
            break
else:
    print("can't open video")            # 캡쳐 객체 초기화 실패
cap.release()                            # 캡쳐 자원 반납
cv2.destroyAllWindows()

FPS: 29.970030, Delay: 33ms


KeyboardInterrupt: 

카메라로부터 읽은 영상이 너무 고화질인 경우 픽셀 수가 많아 연산하는데 시간이 많이 걸리는 경우가 있습니다. 이때 프레임의 폭과 높이를 제어해서 픽셀 수를 줄일 수 있다.

In [34]:
cap = cv2.VideoCapture(0)                         # 카메라 0번 장치 연결
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)         # 프레임 폭 값 구하기
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)       # 프레임 높이 값 구하기
print('Original width: %d, height: %d'%(width, height))

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 320)            # 프레임 폭을 32O으로 설정
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)           # 프레임 높이를 240으로 설정
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)         # 재지정한 프레임 폭 값 구하기
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)       # 재지정한 프레임 높이 값 구하기
print('Resized width: %d, height: %d'%(width, height))

if cap.isOpened():                                # 객체 초기화
    while True:
        ret, img = cap.read()                     # 영상 프레임 읽기
        if ret:
            cv2.imshow('camera', img)
            if cv2.waitKey(1) != -1:
                break
        else:
            print('no frame')
            break
        
else:
    print("can't open camera!")
    
cap.release()
cv2.destroyAllWindows()

Original width: 640, height: 480
Resized width: 320, height: 240


_________________________________________

### 카메라로 사진 찍기
하나의 프레임만 촬영

In [40]:
cap = cv2.VideoCapture(0)

if cap.isOpened():   # 파일이나 카메라 장치에 제대로 연결되어 있는지 확인
    while True:
        ret, frame = cap.read()  # 프레임을 읽었는지 안 읽었는지
        if ret:
            cv2.imshow('camera', frame)
            if cv2.waitKey(1) != -1:          # 아무 키를 누르면
                cv2.imwrite('photo.jpg', frame)
                break
        else:
            print('no frame!')
else:
    print('no camera!')
cap.release()
cv2.destroyAllWindows()      

**여러 프레임을 동영상으로 저장**  
cv2.VideoWriter(file_path, fourcc, fps, (width, height)): 비디오 저장 클래스 생성자 함수
- file_path: 비디오 파일 저장 경로
- fourcc: 비디오 인코딩 형식 4글자
- fps: 초당 프레임 수
- height, width: 프레임 폭과 프레임 높이
- writer: 생성된 비디오 저장 객체

writer.write(frame): 프레임 저장
- frame: 저장할 프레임, Numpy 배열

writer.set(id, value): 속성 변경  
writer.get(id): 속성 변경  
ret = writer.fourcc(c1, c2, c3, c4): fourcc 코드 생성
- c1, c2, c3, c4: 인코딩 형식 4글자, 'MJPG', 'DIVX' 등
- ret: fourcc 코드

cv2.VideoWRiter_fourcc(c1, c2, c3, c4): cv2.VideoWriter.fourcc()

In [42]:
cap = cv2.VideoCapture(0)
if cap.isOpened():
    file_path = './record.avi'
    fps = 25.40
    fourcc = cv2.VideoWriter_fourcc(*'DIVX')               # 인코딩 포맷 문자
    width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    size = (int(width), int(height))                       # 프레임 크기
    out = cv2.VideoWriter(file_path, fourcc, fps, size)    # VideoWriter 객체 생성
    while True:
        ret, frame = cap.read()
        if ret:
            cv2.imshow('camer', frame)
            out.write(frame)                               # 파일 저장
            if cv2.waitKey(int(1000/fps))  != -1:
                break
        else:
            print('no frame!')
            break
    out.release()                                          # 파일 닫기 
else:
    print("can't open camera")
cap.release()
cv2.destroyAllWindows()    